# 前処理（続き）
## 単語境界の修正
### 単語境界誤り
形態素解析では好ましくない単語分割結果が得られることがある。

- 固有名詞の分割 New York → new|york
- 固有名詞の解析ミス はがない（僕は友達が少ない） → はが|ない
- 解析単位が小さすぎる おすし → お|すし
- 解析単位の流儀の違い きれいな → きれい|な

### 問題点

- 解析ミス（違う単語として認識されている）は自明に問題
- 「形態素|解析」についての文書と「複素|解析」についての文書の類似度が変に高くなる

### 解決策

- 形態素解析の辞書を変える: Neologd
  - はてなキーワードなどWebから収集した新語を収録
  - 週2回更新
  - 固有名詞が充実
  - 一般名詞が誤って固有名詞と検出される気がする（体感）

- 細かすぎる解析単位をくっつける
  - 複合名詞: 連続する名詞をくっつける
    - フォーマルな文章では結構うまくいく
    - 口語的な文章だと助詞を省くために過剰に結合される 
      - e. 猫は魚を食べた → 猫魚食べた
  - ルールベースでくっつける
    - 形態素解析で品詞を調べて，「接頭辞」「非自立」とかを処理する
    - ルールを作るのがしんどい
  - 固有表現認識器CaboChaやKNPに投げる
    - 上述の単純な方法と比べて遅そう
  - コーパスから良さそうな分割を学習
    - [隠れマルコフ過程](https://ipsj.ixsq.nii.ac.jp/ej/?action=pages_view_main&active_action=repository_view_main_item_detail&item_id=15470&item_no=1&page_id=13&block_id=8)
    - ある単語と共起し，それ以外であまり共起しない単語を検出 ([Word2Vec](http://papers.nips.cc/paper/5021-distributed-representations-of-words-andphrases))

### 補足
- 分割前の情報が失われる
  - 「形態素解析」≠「形態素」+「解析」だが，「形態素」+「解析」は「りんご」よりは「形態素解析」に近いはず
- 単語のエントリ数が増える・出現頻度の低い単語ができる
  - 単語の組み合わせなので
  - エントリ数増加→メモリ消費・計算量
  - 出現頻度が減る→教師データにするのに不十分

## ストップワード除去
### ストップワード

ストップワードとは，処理結果に影響しない・悪影響を与えるという理由で除去することが望ましい単語のこと。

### 問題点

- 「Y of X」で検索をして「of」をたくさん含む文書がヒットしても困る
- 計算量

### 解決策

- ストップワード一覧を作り，それを機械的に除去
  - [MySQL 全文探索](http://dev.mysql.com/doc/refman/5.1/ja/fulltext-stopwords.html)
- 単語の出現頻度や品詞情報から判断

## TF-IDF

### 概要

TF-IDFとは，文書集合内の文書について，特徴的な単語を検出する尺度。ヒューリスティック。
TF (Term Frequency) と IDF (Inverse Document Frequency) の積。

\begin{align*}
\mathrm{tfidf_{w,d}} &= \mathrm{tf_{w,d}} \cdot  \mathrm{idf_{w}}\\
\mathrm{tf_{w,d}} &= \frac{n_{w,d}}{\sum_{w'} n_{w',d}}\\
\mathrm{idf_w} &=  \log \frac{|D|}{|\{d'\mid d'\in D \land w \in d'\}|}
\end{align*}

文書$d$における，単語$w$のTF-IDFを考える。

TFは，$w$の$d$内の出現頻度である。単語$w$が文書$d$に出現する回数を$n_{w, d}$とすると，
\begin{align*}
\mathrm{tf_{w,d}} &= \frac{d\text{中}w\text{出現回数}}{d\text{中全単語数}}\\
&=\frac{n_{w,d}}{\sum_{w'} n_{w',d}}
\end{align*}

IDFは，全文書のうち$w$を含む文書の割合の逆数の対数である。全文書の集合を$|D|$とする。全文書のうち$w$を含む文書の集合は
$$\{d'\mid d'\in D \land w \in d'\}$$
よって，全文書のうち$w$を含む文書の割合は，
$$\frac{|\{d'\mid d'\in D \land w \in d'\}|}{|D|}$$
逆数の対数をとって，
$$\mathrm{idf_w} =  \log \frac{|D|}{|\{d'\mid d'\in D \land w \in d'\}|}$$
全文書中の単語の頻度ではないことに注意。

TFとIDFの積がTF-IDFである。
\begin{align*}
\mathrm{tfidf_{w,d}} &= \mathrm{tf_{w,d}} \cdot  \mathrm{idf_{w}}\\
&=\frac{n_{w,d}}{\sum_{w'} n_{w',d}}\log \frac{|D|}{|\{d'\mid d'\in D \land w \in d'\}|}
\end{align*}

定性的には，「$d$に何度も出てくる単語は特徴的」「どの文書にもあまねく存在する単語は特徴的でない」という感じ。

さらに文書の単語数を考慮した[Okapi BM25](https://ja.wikipedia.org/wiki/Okapi_BM25)というものもある。

### TF-IDFの計算

文書集合が決まっていればIDFは一定なので，IDFデータが配布されていることも。
- [Wikipedia日本語IDF辞書](https://catindog.hatenablog.com/entry/2017/02/15/222915)


「$d$を含む文書の割合」を「全文書中の$d$の頻度」で近似すると，IDFは全文書中の$d$の頻度で定まる。[単語の出現頻度のデータ](https://www.kaggle.com/rtatman/english-word-frequency)があればコーパスを用意せずにIDFを計算できる。  
さらに，単語の出現頻度は[Zipfの法則](https://ja.wikipedia.org/wiki/ジップの法則)に従うので，単語の出現頻度はその単語が$n$番目に出現頻度の高い単語とすると，出現頻度は$\displaystyle \frac1c$に比例する（近似）。[出現頻度順単語リスト](https://www.wordfrequency.info/free.asp)があればそこから出現頻度を推定し，さらにIDFを推定することができる。


Twitterのような短い文書の集合では文書が同じ単語を2回以上含むことが少ないので，「全文書中の$d$の頻度」と「$d$を含む文書の割合」があまり変わらないため，「全文書中の$d$の頻度」をIDFとしても良さそう。あるいはツイート1つを文書にするのではなく，ユーザーごとにツイートをまとめて1つの文書とする（文書@pizzacat83, 文書@TwitterJP, 文書@RealDonaldTrump,...）と面白いかも…？

### 応用

- IDF値が低い（どの文書にもあまねく含まれている）単語をストップワードとして除去
- 単語の意味情報から文書全体の意味情報を推定するときに，TF-IDFやIDFを使って単語に重み付け

## 分散表現
### 概要

ニューラルネットワーク等ではベクトルを入力に取るので，自然言語処理では単語をベクトルで表す必要がある。文字列としての単語は可変長データであり扱いにくい上，コンピュータにとってそのバイト列はあまり意味を持たない。  
そこで文字列をハッシュなどに食わせて，IDを振る。

gensimというPythonの自然言語処理パッケージがいい感じのクラス[`Dictionary`](https://radimrehurek.com/gensim/corpora/dictionary.html)を用意している。

In [ ]:
from gensim import corpora

words = ['I ate the delicious sushi .'.split()]

dictionary = corpora.Dictionary(words)
print(dictionary.token2id)

この数値自体には特に意味はないので，これをそのまま入力にしてしまうと意味のない数値に対して四則演算することになり，あんまりいい結果が得られなさそう。そこで **One-hot vector** というものを使う。

One-hot vectorとは，IDが$i$であれば$i$番目のみ1, それ以外の要素を0としたベクトルである。

$$v_{sushi} = (0, 0, 0, 0, 1, 0)$$

これであれば`sushi`と`I`の差と`sushi`と`delicious`の差が等しい（任意の単語ペアに対し等しい）ので，さっきよりマシ。ただ，次元数が総単語数であり，日本語は2万語くらいありそうだしかな漢字の表記揺れも考えるとヤバそう…。

そもそも要素のうち1つだけ1でそれ以外0，というベクトルはめちゃくちゃ冗長なので圧縮をしましょう，というのが分散表現である。分散表現では300次元程度の小さなベクトルで，単語の意味が近い単語はベクトルも近くなるように成分を設定する。一番有名な分散表現アルゴリズムは **Word2Vec** で，2層のニューラルネットワークからなる。今日は理論には立ち入らず，学習済みのモデルで遊ぼう。

### 学習済み単語ベクトル

ドメインに合わせたベクトルが欲しいとか，精度を気にするのでチューニングしたいとかでなければ，マシンパワー有り余ってる人たちが作った学習済み単語ベクトルを使うのが手っ取り早くて楽。

https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('wikipedia.wv', binary=True)

まずは，似ている単語が近いベクトルを持っていることを確認しよう。引数の単語に対応するベクトルと近いベクトルを持つ単語10個が表示される。

In [ ]:
model.similar_by_word("寿司")

Word2Vecは加法構成性といって，意味の足し引きがベクトルの足し引きに対応するという性質もある。意味の表現として有用な性質。

In [ ]:
# 寿司 - 魚 + 肉 = ?
model.most_similar(positive=["寿司", "肉"], negative=["魚"])

In [ ]:
# 有名なやつ
model.most_similar(positive=["王", "女"], negative=["男"])

In [ ]:
# 文法的な関係の足し引きもできる
# 最長 - 長い + 高い
# 英語だと，did - do + eat -> ate
model.most_similar(positive=["最長", "高い"], negative=["長い"])

### 単語の意味から文の意味へ

- Bag of Words
  - 単語の順序は無視するが，個数は無視しない
- 単語ベクトルをIDFで重みつけて足す
- Doc2Vec